In [1]:
import os, pickle, random
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter

In [2]:
os.chdir('..')
from src.models.genetic_algorithm import *
from src.models.train_animation_predictor import *
from src.models import config
from src.models.ordinal_classifier_fnn import *
from src.models.genetic_algorithm import create_animation_vector
from src.animations.insert_animation import create_animated_svg
from src.models.entmoot_functions import *

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1. Load test data for evaluation

In [4]:
test_data = pd.read_csv('data/path_selector/path_selector_test.csv')

In [5]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg
0,logo_147,10,0.288970,-7.284798,1.094182,-4.437811,-0.352077,-0.933593,-0.200254,0.931484,0.506142,0.100814,0,1.0,1.0,157,157,156,157,157,156,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,0.017877,0.111550,0.168001,0.084745,13
1,logo_147,0,-2.807291,-11.223132,3.151704,2.541029,-0.119346,0.909742,2.923519,-1.477561,0.059558,-1.025471,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.077887,0.707306,0.038944,0.562185,13
2,logo_147,1,0.869143,-11.525720,-0.201577,3.906369,-2.661400,-0.757390,-0.195925,-0.380663,-0.033850,0.286074,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.026691,0.467979,0.161487,0.450085,13
3,logo_147,12,9.519547,-1.598271,-4.577534,-2.522939,-0.754545,-0.054362,-1.478206,-1.002636,-1.703539,-0.243925,0,1.0,1.0,22,65,148,22,65,148,136.230769,-114.230769,142.846154,-77.846154,154.769231,-6.769231,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,0.057923,0.271332,0.502114,0.135666,13
4,logo_147,2,-0.253583,-11.400860,-0.170891,5.689338,-0.972701,1.972220,-0.119872,0.083583,0.253451,-0.066204,0,1.0,1.0,157,157,156,0,0,0,136.230769,20.769231,142.846154,14.153846,154.769231,1.230769,15.461538,-15.461538,22.076923,-22.076923,34.769231,-34.769231,0.050151,0.477448,0.113555,0.445340,13


In [6]:
test_data = retrieve_m1_predictions(test_data)

2021-05-09 23:32:00::INFO::Number of animated paths: 448/1892


In [7]:
test_data = retrieve_animation_midpoints(test_data, drop=True)

2021-05-09 23:32:00::INFO::Non-animated paths won't be used for training
2021-05-09 23:32:00::INFO::Start extraction midpoint of animated paths as feature


In [8]:
scaler = pickle.load(open(config.scaler_path, 'rb'))
test_data[config.sm_features] = scaler.transform(test_data[config.sm_features])

In [9]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations
0,logo_147,10,-0.614143,-1.434997,0.378169,-0.989337,0.380872,-1.097045,-0.385497,0.361610,0.302239,0.057947,0,1.0,1.0,0.015112,0.313663,0.464383,157,157,156,0.226906,-0.164612,0.441620,-0.020036,0.690207,-0.040372,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,-1.041299,-0.830271,-1.036934,-1.711635,-0.592006,1,"[10, 12, 11]","(0.0428012672222377, 0.08474520367214555)",-1.874613,-1.839426
1,logo_147,12,0.583193,-0.443299,-0.727442,-0.609706,0.236960,-0.761934,-1.199034,-0.492079,-0.984769,-0.223232,0,1.0,1.0,-1.298036,-0.736364,0.376969,22,65,148,0.226906,-1.582235,0.441620,-1.161878,0.690207,-0.141752,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.866886,-0.235145,0.395292,-1.454827,-0.592006,1,"[10, 12, 11]","(0.923646143983946, 0.13566598867571875)",1.678236,-1.612458
2,logo_147,11,0.467990,-1.013074,-0.523382,1.332578,0.302889,-0.325943,-1.633460,-0.478933,-0.650176,-0.719644,0,1.0,1.0,-1.298036,-0.736364,0.376969,22,65,148,0.226906,-1.582235,0.441620,-1.161878,0.690207,-0.141752,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.329837,1.732380,0.019159,0.887998,-0.592006,1,"[10, 12, 11]","(0.6923176974460455, 0.6002088945082733)",0.745183,0.458138
3,logo_148,11,-0.801327,-1.728030,0.653964,0.792801,-0.357631,-1.769588,-0.790681,0.578952,-1.516484,-0.041647,0,1.0,1.0,0.015112,0.313663,0.464383,0,0,0,-1.023478,0.832398,-0.540434,0.783018,0.605205,0.030927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.030794,0.886960,-1.221488,-0.695136,-0.536257,1,[11],"(0.4999999999999999, 0.5)",-0.030522,0.011479
4,logo_149,11,0.112664,0.296510,-0.925415,0.834943,-0.866570,-1.698705,-1.346968,1.367504,-2.451012,-0.763492,0,1.0,1.0,0.015112,0.313663,0.464383,0,0,0,-1.023478,0.832398,-0.540434,0.783018,0.605205,0.030927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.511334,0.055165,0.388339,-1.258280,-0.536257,1,"[11, 12, 13]","(0.650810561967709, 0.2862981256047344)",0.577766,-0.941049


In [10]:
test_data.to_csv('data/entmoot_optimization/evaluation_data.csv', index=False)

## 2. Apply Entmoot optimizer to test data

In [4]:
# Set seeds in order to reproduce results
torch.manual_seed(73)
random.seed(73)
np.random.seed(73)

In [5]:
# Load test data for checking evaluation procedure
test_data = pd.read_csv('data/entmoot_optimization/evaluation_data.csv')
path_vectors = test_data[config.sm_features].values.tolist()

In [6]:
with open("models/entmoot_optimizer_final.pkl", "rb") as f:
    optimizer = pickle.load(f)

In [7]:
# Load surrogate model for function evaluations
func = SurrogateModelFNN()

In [8]:
# Predict animation vectors
an_vec_preds = []
score_preds = []
for i in range(len(path_vectors)):
    opt_x, opt_y, _ = entmoot_predict(optimizer, func, path_vectors[i], n_calls=1)
    an_vec_preds.append(opt_x)
    score_preds.append(-opt_y)
    if (i+1) % 10 == 0:
        print(f'Finished animation prediction for path {i+1}.')

Academic license - for non-commercial use only - expires 2021-06-18
Using license file /Users/jani/gurobi.lic
Finished animation prediction for path 10.
Finished animation prediction for path 20.
Finished animation prediction for path 30.
Finished animation prediction for path 40.
Finished animation prediction for path 50.
Finished animation prediction for path 60.
Finished animation prediction for path 70.
Finished animation prediction for path 80.
Finished animation prediction for path 90.
Finished animation prediction for path 100.
Finished animation prediction for path 110.
Finished animation prediction for path 120.
Finished animation prediction for path 130.
Finished animation prediction for path 140.
Finished animation prediction for path 150.
Finished animation prediction for path 160.
Finished animation prediction for path 170.
Finished animation prediction for path 180.
Finished animation prediction for path 190.
Finished animation prediction for path 200.
Finished animation 

In [9]:
test_data['animation_vector'] = an_vec_preds

In [10]:
test_data['reward'] = score_preds

In [11]:
test_data.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations,animation_vector,reward
0,logo_147,10,-0.614143,-1.434997,0.378169,-0.989337,0.380872,-1.097045,-0.385497,0.361610,0.302239,0.057947,0,1.0,1.0,0.015112,0.313663,0.464383,157,157,156,0.226906,-0.164612,0.441620,-0.020036,0.690207,-0.040372,15.461538,141.538462,22.076923,134.923077,34.769231,121.230769,-1.041299,-0.830271,-1.036934,-1.711635,-0.592006,1,"[10, 12, 11]","(0.0428012672222377, 0.08474520367214555)",-1.874613,-1.839426,"[0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",2
1,logo_147,12,0.583193,-0.443299,-0.727442,-0.609706,0.236960,-0.761934,-1.199034,-0.492079,-0.984769,-0.223232,0,1.0,1.0,-1.298036,-0.736364,0.376969,22,65,148,0.226906,-1.582235,0.441620,-1.161878,0.690207,-0.141752,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.866886,-0.235145,0.395292,-1.454827,-0.592006,1,"[10, 12, 11]","(0.923646143983946, 0.13566598867571875)",1.678236,-1.612458,"[0, 0, 0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",3
2,logo_147,11,0.467990,-1.013074,-0.523382,1.332578,0.302889,-0.325943,-1.633460,-0.478933,-0.650176,-0.719644,0,1.0,1.0,-1.298036,-0.736364,0.376969,22,65,148,0.226906,-1.582235,0.441620,-1.161878,0.690207,-0.141752,15.461538,6.538462,22.076923,42.923077,34.769231,113.230769,-0.329837,1.732380,0.019159,0.887998,-0.592006,1,"[10, 12, 11]","(0.6923176974460455, 0.6002088945082733)",0.745183,0.458138,"[0, 0, 0, 0, 1, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",2
3,logo_148,11,-0.801327,-1.728030,0.653964,0.792801,-0.357631,-1.769588,-0.790681,0.578952,-1.516484,-0.041647,0,1.0,1.0,0.015112,0.313663,0.464383,0,0,0,-1.023478,0.832398,-0.540434,0.783018,0.605205,0.030927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.030794,0.886960,-1.221488,-0.695136,-0.536257,1,[11],"(0.4999999999999999, 0.5)",-0.030522,0.011479,"[0, 0, 0, 0, 1, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",2
4,logo_149,11,0.112664,0.296510,-0.925415,0.834943,-0.866570,-1.698705,-1.346968,1.367504,-2.451012,-0.763492,0,1.0,1.0,0.015112,0.313663,0.464383,0,0,0,-1.023478,0.832398,-0.540434,0.783018,0.605205,0.030927,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.511334,0.055165,0.388339,-1.258280,-0.536257,1,"[11, 12, 13]","(0.650810561967709, 0.2862981256047344)",0.577766,-0.941049,"[0, 0, 0, 0, 1, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",3


In [12]:
test_data['reward'].mean()

2.497767857142857

In [13]:
test_data['reward'].value_counts()

2    235
3    191
4     18
1      4
Name: reward, dtype: int64